2022년 5월 동아리에서는 신규 기수를 대상으로 [분류 모델 경진대회](https://cafe.naver.com/pnuiba/115)를 개최하였습니다. 동아리 회장으로서 대회를 기획하고 운영하였으며, 오늘은 대회 운영에 사용했던 코드를 정리하려고 합니다.\
\
DACON에서 진행하는 대회와 동일한 방식으로 경진대회를 진행하였으며, 이를 위해 데이터 셋을 만들고 성과를 평가하기 위해 아래와 같이 코드를 작성하였습니다.

# 1. train/test 데이터 셋 만들기

대회에서 사용한 데이터는 UCI의 [adult](https://archive-beta.ics.uci.edu/dataset/2/adult) 데이터로, 인구 데이터를 바탕으로 소득이 5만 달러 이하인지 초과인지를 예측하는 데이터입니다. 먼저 UCI에서 주어진 데이터를 대회에서 사용할 train과 test 데이터로 만들었습니다.

In [1]:
import pandas as pd
import urllib

# 데이터 불러오기
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
headers = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship',
        'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'target']

df = pd.read_csv(urllib.request.urlopen(url), header=None, names=headers)
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


대회 과정에서 원본 데이터를 찾아 target을 확인하지 못하도록, 데이터를 섞고 index를 재설정하였습니다. 그리고 문자형으로 되어있는 target을 편의상 숫자형으로 변환하였습니다.

In [2]:
# 데이터를 랜덤하게 정렬
df = df.sample(frac=1, random_state=1)
# index 재설정
df = df.reset_index(drop=True)

# target을 숫자형으로 변환
df['target'] = df['target'].apply(lambda x : 0 if x==' <=50K' else 1)

df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,62,Self-emp-not-inc,26911,7th-8th,4,Widowed,Other-service,Not-in-family,White,Female,0,0,66,United-States,0
1,18,Private,208103,11th,7,Never-married,Other-service,Other-relative,White,Male,0,0,25,United-States,0
2,25,Private,102476,Bachelors,13,Never-married,Farming-fishing,Own-child,White,Male,27828,0,50,United-States,1
3,33,Private,511517,HS-grad,9,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,0
4,36,Private,292570,11th,7,Never-married,Machine-op-inspct,Unmarried,White,Female,0,0,40,United-States,0


다음으로 train_test_split을 활용하여 train 데이터셋과 test 데이터셋을 만들었습니다. train 데이터는 X_train과 y_train을 merge 하여 train data로 만들었습니다. X_test는 test 데이터로 제공하고, y_test는 answer로 저장하여 운영진인 저만 가지고 있었습니다.

In [3]:
from sklearn.model_selection import train_test_split

X = df.drop('target', axis=1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=.25, random_state=1)

train_data = X_train.merge(y_train, left_index=True, right_index=True)
train_data.to_csv('train.csv')
X_test.to_csv('test.csv')
y_test.to_csv('answer.csv')

대회가 진행될 때 참가자는 중간 성과를 확인할 수 있었습니다. 이때, test 데이터 전체를 사용한 성과를 알려주지 않고 전체의 50%를 사용한 성과를 안내하였습니다. 50%로 성과를 안내하는 이유는 테스트 데이터에 과적합이 되지 않도록 하기 위함입니다. 이에 필요한 답안지를 y_test의 50%를 랜덤추출하여 만들었습니다.

In [4]:
answer_50 = y_test.sample(frac=.5, random_state=1)
answer_50.to_csv('answer_50.csv')

# 2. 성과 평가를 위한 함수 만들기

데이터셋을 구성한 후, 성과 평가를 위한 함수를 작성하였습니다. 대회 진행 중에 사용할 중간 평가와 최종 평가를 하나의 함수로 구현하였습니다.\
\
대회 참가자에게는 몇 가지 규칙을 안내하였습니다.
- 컬럼명은 반드시 pred로 설정
- index를 포함하여 저장
- test 데이터 삭제 금지

이러한 규칙이 지켜졌는지 확인하기 위해 먼저 데이터에 문제가 있는지 확인하였으며, 오류가 없는 데이터를 대상으로 성과를 출력하는 코드를 작성하였습니다.

대회에 사용한 원본 데이터가 범주 불균형 문제가 있었기 때문에, accuracy가 아닌 f1-score를 사용하였습니다.

In [5]:
from sklearn.metrics import f1_score

def IBA_contest_result(df_pred):
    while True:
        # 제출 답안의 오류 검증
        if len(df_pred) != 8141:    # test 데이터의 길이 == 8141
            print('# 오류 : 제출 답안의 길이가 일치하지 않음')
            break
            
        if df_pred.isna().any().any():
            print('# 오류 : 제출 답안에 결측치 존재')
            break
            
        if 'pred' not in(df_pred.columns):
            print('# 오류 : 제출 답안에 pred 컬럼이 없음')
            break
            
        mode = input('1 - 중간 성과 /// 2 - 최종 성과 ==> ')
        if mode == '1':
            mode = '중간 평가'
            answer = pd.read_csv('answer_50.csv', index_col=0)
        
        elif mode =='2':
            mode = '최종 성과'
            answer = pd.read_csv('answer.csv', index_col=0)
            
        else:
            print('# 오류 : mode 입력값 오류')
            break
        
        result = answer.merge(right=df_pred, how='left', left_index=True, right_index=True)
        
        if result.isna().any().any():
            print('# 오류 => 예측 파일의 index 오류')
            break
            
        f1 = f1_score(result['target'], result['pred'])
        print(f'\n##### {mode} #####\n')
        print(f'f1_score ==> {f1:.4f}')
        
        break

함수를 사용하는 방법은 제출 받은 파일을 불러와서 함수에 넣어주기만 하면 됩니다.

In [6]:
# 제출 답안 불러오기
fpath = 'example.csv'
df_pred = pd.read_csv(fpath, index_col=0)

# 성과 평가
IBA_contest_result(df_pred)

1 - 중간 성과 /// 2 - 최종 성과 ==> 1

##### 중간 평가 #####

f1_score ==> 0.3230


이러한 프로그램을 사용하여 대회를 성공적으로 마칠 수 있었습니다. 참가자 입장이 아닌 운영자의 입장에서 대회를 바라볼 수 있는 좋은 경험이었습니다.\
대회를 진행하기 위해 다른 방법이 아닌 스스로 프로그램을 만들어 대회를 기획한 것이 힘들어도 유익한 경험이었던 것 같습니다.\
혹시 소모임이나 동아리 내 자체 공모전을 기획하시는 분들께 제 글이 도움이 되었길 바라며, 오늘은 여기서 마무리하도록 하겠습니다. 읽어주셔서 감사합니다.